In [1]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"]= "1"  # Set the GPUs to use
import torch
print('Current cuda device:', torch.cuda.current_device())
print('Count of using GPUs:', torch.cuda.device_count())

Current cuda device: 0
Count of using GPUs: 1


In [2]:
import os
import re
import random
import warnings

import pandas as pd
import numpy as np
import torch

from transformers import (
    AutoTokenizer, AutoModelForSeq2SeqLM, 
)
from datasets import Dataset

warnings.filterwarnings('ignore')

In [3]:
MODEL_CHECKPOINT = 'training_results/v1/ainize_kobart_news_v1/checkpoint-2535'
DATA_PATH = 'data/original/train.csv'

In [4]:
data_df = pd.read_csv(DATA_PATH)

dataset = Dataset.from_pandas(data_df).shuffle(seed=100).train_test_split(0.1)
train_dataset = dataset['train']
eval_dataset = dataset['test']

data_df = eval_dataset.to_pandas()

In [5]:
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT)
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

In [6]:
idx = random.randint(0, len(data_df)-1)

sentence = data_df.context.iloc[idx]

inputs = tokenizer(sentence, max_length=1026, truncation=True, return_tensors="pt")
inputs = {k: v.cpu() for k, v in inputs.items() if k != 'token_type_ids'}
print(inputs['input_ids'].shape)

logits = model(**inputs).logits
print(logits.shape)

keys = logits.argmax(-1).squeeze().tolist()

tokenizer.decode(keys)

torch.Size([1, 1026])
torch.Size([1, 1026, 30000])


'<s>음성정 제12항, 음성군 장애인가족지원센터 설치 및 민간위탁 동의안은 상정했으며,</s>생활지원과 이하 장애셔서 장애설명. 주시기 바랍.</s>생활지원과장 이하 장애군 장애인가족지원센터 설치 및 민간위탁 동의안은 대해 설명 함겠습니다.</s> 음성건명은 음성군 장애인가족지원센터 설치 및 민간위탁 동의안이며, 근거법령은 <「음성군 장애인가족 지원 조례」 제7조, < <음성군 사무의 민간위탁 촉진 및 관리 조례」제4조에 근거하고 있</s>이유,</s> 음성군이 제공하는 장애인복지서비스는 주로 성인 및 장애인 당사자에게만 집중되어 있어 학령기 장애아동은 물론 장애인가족들에게 현실성 있는 사례관리 및 프로그램을 적절하게 제공할 수 있는 센터 설치가 요구되는 사항이며,</s>인가족지원센터센터는 관내 재가인가족의 복지서비스에 대한 욕구를 오랜 기간 동안 검토하고 수용한 사업으로서 관내 재가 장애인들의 지원을 위해 관내 재가단체와 연계하여 장애인가족에게 체계적인 사례관리를 제공할 것이며, 센터를 효율적으로 운영할 수 있는 비영리법인 및 장애인단체를 공개모집하여 위탁운영하게 함으로써 장애인 당사자는 물론 장애인가족의 자립을 돕고 장애인의 소외감을 박탈감 해소에 기여할 수 있도록 것으로</s> 장애군의 장애 장애인가족의에 관한 장애적 자문을 구하고 센터의 효율적인 설치 운영을 위하여 장애 같은 절차와 내용으로 장애인가족지원센터를 설치 및위탁하고자 음성군의회의 동의를 받고자 함 사항.</s> 번째, 설치 및위탁 주요 내용.</s> 사업개요 및 운영의</s>의 주요치는 음성군 금왕읍 일원에서며, 사무실은 임차로 하고, 사업기간은 2017년 1월부터 계획하고 있으며, 총사업비는 전액 1억 7,438만 7천원으로 전액 군비이며,</s>방식은 민간위탁금모집 및 운영 이용대상은 장애인복지법에 의한 장애 및 그 가족.으며,</s>경위비는</s> 10월 29일 음성군 장애인가족 지원조례가 제정된 이후에 2016년 8월 29일 충주시 장애인가족지원센터 벤치마킹을 하였고 금년 10월 1

# Generate

In [7]:
idx = random.randint(0, len(data_df)-1)
sample = data_df.context.iloc[idx]
y = data_df.summary.iloc[idx]
sample

'의사일정 제8항, 생활폐기물 수집ㆍ운반 사무 민간위탁 계획안 동의의 건을 상정합니다. 환경보호과장님께서는 나오셔서 안건에 대하여 제안설명하여 주시기 바랍니다. 환경보호과장입니다. 의안번호 제9호로 제출한 생활폐기물 대행업체 민간위탁계획안에 대하여 설명드리겠습니다. 2페이지가 되겠습니다. 생활폐기물 수집ㆍ운반 사무 민간위탁 계획안입니다. 「폐기물관리법」에 의거 관내에서 발생하는 생활폐기물을 수집ㆍ운반처리하여야 하며, 생활폐기물을 위생적이고 안정적으로 수집ㆍ운반ㆍ처리하기 위하여 조례가 정하는 바에 따라 대행할 수 있으며 청소대행업무 민간위탁에 대하여 음성군 사무의 민간위탁촉진 및 관리조례에 의거 음성군의회의 동의를 얻고자 하는 사항이 되겠습니다. 생활폐기물 수집ㆍ운반처리 대행에 대한 민간위탁 현황을 말씀을 드리면, 생활폐기물 청소대행은 정부의 방침에 따라 1999년부터 민간위탁으로 현재까지 운영 관리하고 있는 실정이 되겠습니다. 생활폐기물 청소대행업체 현황은 문화환경은 음성, 소이, 원남, 중부환경은 맹동, 삼성, 생극, 감곡, 주식회사 음성환경은 금왕, 주식회사 대소환경개발은 대소로 4개 구역으로 나누어서 4개 업체가 운영 관리하고 있습니다. 청소대행 민간위탁계획으로 우리 군은 현재 생활폐기물 청소대행을 주고 있는 실정으로 「음성군 사무의 민간위탁 촉진 및 관리조례」를 제정하였으며, 2010년도 충청북도 행정감사결과 청소대행업무에 대하여 의회의 동의를 득하지 아니하여 지적을 받은 사례가 있습니다. 이에 「폐기물관리법」 및 「음성군사무의 민간위탁 촉진 및 관리조례」제4조제3항에 의거 생활폐기물 수집ㆍ운반사무의 민간위탁에 대하여 음성군의회의 동의를 얻고자 하는 사항이 되겠습니다. 끝으로 제안자의견입니다. 생활폐기물 청소업무는 당초 정부의 민간위탁방침 및 총액인건비제 시행으로 군에서 직영하기에는 방대한 사무이며, 당초 청소업무를 민간위탁할 때 군에서 보유한 인력 및 청소차량을 청소대행업체에 이관하였고, 현재 운영하고 있는 실정과 같이 청소업무를 민간에 위탁함이 타당

In [8]:
max_input_length = 1026

input_ids, _, _ = tokenizer(sample, max_length=max_input_length, truncation=True, return_tensors="pt").values()
input_ids = input_ids.cpu()

## Greedy Search

In [9]:
outputs = model.generate(input_ids, max_length=512)
y_hat = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print('정답: ', y)
for sentence in y_hat:
    print('예측: ', sentence)

정답:  생활폐기물 수집, 운반 사무 민간위탁 계획안 동의의 건은 현재와 같이 생활폐기물 청소업무를 민간에 위탁하는 것에 대한 동의를 구하기 위해 제안됨. 해당 안건은 가결됨.
예측:  생활폐기물 수집, 운반 사무 민간위탁 계획안은 <폐기물관리법>에 의거 관내에서 발생하는 생활폐기물을 수집, 운반 처리해야 함. 생활폐기물을 위생적이고 안정적으로 수집, 운반, 처리하기 위하여 조례에 따라 대행할 수 있으며 청소대행업무 민간위탁에 대하여 음성군의회의 동의를 얻고자 제안됨. 해당 안건은 가결됨.


## Beam Search

In [10]:
outputs = model.generate(
    input_ids, max_length=512, 
    num_beams=5, 
    no_repeat_ngram_size=2, 
    num_return_sequences=5, 
    early_stopping=True,
)
y_hat = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print('정답: ', y)
for sentence in y_hat:
    print('예측: ', sentence)

정답:  생활폐기물 수집, 운반 사무 민간위탁 계획안 동의의 건은 현재와 같이 생활폐기물 청소업무를 민간에 위탁하는 것에 대한 동의를 구하기 위해 제안됨. 해당 안건은 가결됨.
예측:  생활폐기물 수집, 운반 사무 민간위탁 계획안에 대한 동의의 건은 가결됨.
예측:  생활폐기물 수집, 운반 사무 민간위탁 계획안에 대한 동의의 건은 채택됨.
예측:  생활폐기물 수집, 운반 사무 민간위탁 계획안에 대한 동의의 건은 가결되었음.
예측:  생활폐기물 수집, 운반 사무 민간위탁 계획안 동의의 건은 폐기물관리법에 의거 관내에서 발생하는 생활 폐기물을 수집 및 반송 처리해야 함. 생활 폐기물을 위생적이고 안정적으로 수집·운반, 처리하기 위하여 조례가 정하는 바에 따라 대행할 수 있으며 청소대행업무를 음성군의회의 동의하고자 제정함. 해당 안건은 가결됨.
예측:  생활폐기물 수집, 운반 사무 민간위탁 계획안 동의의 건은 폐기물관리법에 의거 관내에서 발생하는 생활 폐기물을 수집 및 반송 처리해야 함. 생활 폐기물을 위생적이고 안정적으로 수집·운반, 처리하기 위하여 조례가 정하는 바에 따라 대행할 수 있으며 청소대행업무를 음성군의회의 동의하고자 제안함. 해당 안건은 가결됨.


## Sampling

In [11]:
outputs = model.generate(
    input_ids, max_length=512, 
    do_sample=True, 
    top_k=0,
    temperature=0.7,
)
y_hat = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print('정답: ', y)
for sentence in y_hat:
    print('예측: ', sentence)

정답:  생활폐기물 수집, 운반 사무 민간위탁 계획안 동의의 건은 현재와 같이 생활폐기물 청소업무를 민간에 위탁하는 것에 대한 동의를 구하기 위해 제안됨. 해당 안건은 가결됨.
예측:  생활폐기물 수집, 운반 사무 민간위탁 계획안은 <폐기물관리법>에 의거 관내에서 발생하는 생활폐기물을 수집, 운반하고, 생활폐기물을 위생적이고 안정적으로 수집, 운반, 처리하기 위해 조례에 따라 대행할 수 있도록 음성군의회의 동의를 얻고자 제정했으며 현재 민간위탁 업체가 위탁 시 이용되고 있지만 아직도 위탁이 안되어 식별이 안되었기에 해당 안건은 가결됨.


### Top-K Sampling

In [12]:
outputs = model.generate(
    input_ids, max_length=512, 
    do_sample=True, 
    top_k=50,
)
y_hat = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print('정답: ', y)
for sentence in y_hat:
    print('예측: ', sentence)

정답:  생활폐기물 수집, 운반 사무 민간위탁 계획안 동의의 건은 현재와 같이 생활폐기물 청소업무를 민간에 위탁하는 것에 대한 동의를 구하기 위해 제안됨. 해당 안건은 가결됨.
예측:  생활폐기물 수집, 운반 사무 민간위탁 계획안은 <폐기물관리법>에 의거 관내에서 발생하는 생활폐기물을 수집 및 운반하고, 생활폐기물을 위생적이고 안정적으로 수집 및 운반, 처리하기 위하여 조례에 따라 대행할 수 있도록 하고, 청소대행업무 민간위탁에 대하여 음성군의회의 동의를 얻고자 상정함. 해당 안건은 가결됨.


### Top-p (nucleus) sampling

In [13]:
outputs = model.generate(
    input_ids, max_length=512, 
    do_sample=True, 
    top_p=0.92, 
    top_k=0,
)
y_hat = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print('정답: ', y)
for sentence in y_hat:
    print('예측: ', sentence)

정답:  생활폐기물 수집, 운반 사무 민간위탁 계획안 동의의 건은 현재와 같이 생활폐기물 청소업무를 민간에 위탁하는 것에 대한 동의를 구하기 위해 제안됨. 해당 안건은 가결됨.
예측:  생활폐기물 수집, 운반 사무 민간위탁 계획안은 <폐기물관리법>에 의거 관내에서 발생하는 생활폐기물을 수집, 운반 처리해야 함. 생활폐기물을 위생적이고 안정적으로 수집, 운반, 처리하기 위하여 조례에 따라 대행할 수 있으며 청소대행업무 민간위탁에 대하여 음성군의회의 동의를 얻고자 하는 사항이며, 이에 생활폐기물 청소대행업체 현황은 문화환경은 음성, 소이, 원남, 중부환경은 맹동, 삼성, 생극, 감곡, 주식회사 대소환경개발은 대소로 4개 구역으로 나누어서 운영 관리하고자 하는 사항이며, 생활폐기물 청소대행 민간위탁계획안은 음성군 사무의 민간위탁촉진 및 관리조례에 의거 음성군의회의 동의를 얻고자 하는 사항임.


#### top_k = 50 and set top_p = 0.95 and num_return_sequences = 3

In [14]:
outputs = model.generate(
    input_ids, max_length=512, 
    do_sample=True, 
    top_k=50, 
    top_p=0.95, 
    num_return_sequences=5,
)
y_hat = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print('정답: ', y)
for sentence in y_hat:
    print('예측: ', sentence)

정답:  생활폐기물 수집, 운반 사무 민간위탁 계획안 동의의 건은 현재와 같이 생활폐기물 청소업무를 민간에 위탁하는 것에 대한 동의를 구하기 위해 제안됨. 해당 안건은 가결됨.
예측:  생활폐기물 수집, 운반 사무 민간위탁 계획안에 대한 동의의 건은 폐기물관리법에 의거 관내에서 발생하는 생활폐기물을 수집, 운반처리해야하며 생활폐기물을 위생적이고 안정적으로 수집, 운반, 처리하기 위해 조례에 따라 대행할 수 있도록 음성군의회의 동의를 얻고자 상정됨. 해당 안건은 가결됨.
예측:  생활폐기물 수집, 운반 사무 민간위탁 계획안은 <폐기물관리법>에 의거 관내에서 발생하는 생활폐기물을 수집, 운반 처리해야 하고, 생활폐기물을 위생적이고 안정적으로 수집, 운반, 처리하기 위해 조례에 따라 대행할 수 있으며, 청소대행업무 민간위탁에 대하여 음성군의회의 동의를 얻고자 하는 것으로 해당 안건은 가결됨.
예측:  생활폐기물 수집 및 운반 사무 민간위탁 계획안에 대한 동의의 건은 찬성이 가결됨.
예측:  생활폐기물 수집, 운반 사무 민간위탁 계획안은 <폐기물관리법>에 의거 관내 발생하는 생활폐기물을 수집, 운반 처리하고 생활폐기물을 위생적이고 안정적으로 수집, 운반, 처리하기 위하여 조례가 정하는 바에 따라 대행할 수 있도록 음성군 사무의 민간위탁촉진 및 관리조례에 의거 음성군의회의 동의를 얻고자 발의함. 해당 안건은 가결됨.
예측:  생활폐기물 수집, 운반 사무 민간위탁 계획안은 <폐기물관리법>에 의거 관내에서 발생하는 생활폐기물을 수집, 운반 처리하고, 생활폐기물을 위생적이고 안정적으로 수집, 운반, 처리하기 위해 조례에 따라 대행할 수 있으며 청소대행업무 민간위탁촉진 및 관리조례에 의거 음성군의회의 동의를 얻고자 제안됨. 해당 안건은 가결됨.
